In [6]:
# | default_exp classes.DomoApplication

In [2]:
# | exporti
import pandas as pd
from dataclasses import dataclass, field
from typing import Optional
from enum import Enum
import httpx


import domolibrary.routes.application as application_routes
import domolibrary.classes.DomoJob as dmdj
import domolibrary.utils.DictDot as util_dd
import domolibrary.client.DomoAuth as dmda

ModuleNotFoundError: No module named 'domolibrary.routes.job'

# What are applications?
Applications tend to be tooling that domo implemented on the executor framework or extensions to domo products that originated as custom apps that have been fully integrated into the UI.

For example Publish started out as a 3rd party app that was later integrated into domo as an App, and then later received dedicated API paths.

Similarly Remote DomoStats / Observability metrics started out as 3rd party scripting, which was later integrated into Domo's "Executor Framework" an internal framework for managing java / lambda functions, now defined under the application APIs.

Note, pieces of the executor framework has slowly been morphed and adapted into CodeEngine and exposed to end users.

In [ ]:
class WatchDogType(Enum):
    DATASET_ERRORS = "error_detection"
    DATASET_INDEX_TIME = "max_indexing_time"
    LAST_UPDATED_DATA = "last_data_updated"
    ROW_COUNT_CHANGE = "row_count_variance"
    CUSTOM_QUERY = "custom_query"
    OUTLIER_EXECUTION = "execution_variance"


In [ ]:
# | exporti


@dataclass
class DomoApplication:
    id: str
    customer_id: str = None
    name: str = None
    description: str = None
    version: str = None
    execution_class: str = None
    grants: [str] = None
    jobs: [dmdj.DomoJob] = field(default=None)
    jobs_schedule: pd.DataFrame = field(default=None, repr=False)
    auth: dmda.DomoFullAuth = field(repr=False, default=None)

    @classmethod
    def _from_json(cls, obj, auth: dmda.DomoFullAuth = None):
        dd = util_dd.DictDot(obj)

        return cls(
            id=dd.applicationId,
            customer_id=dd.customerId,
            name=dd.name,
            description=dd.description,
            version=dd.version,
            execution_class=dd.executionClass,
            grants=dd.authorities,
            auth=auth,
        )

    @classmethod
    async def get_from_id(cls, auth: dmda.DomoFullAuth, application_id=None):
        res = await application_routes.get_application_by_id(
            application_id=application_id, auth=auth
        )

        if res.status == 200:
            return cls._from_json(obj=res.response, auth=auth)

    async def get_jobs(
        self,
        auth: dmda.DomoFullAuth = None,
        application_id: str = None,
        debug_api: bool = False,
        session: Optional[httpx.AsyncClient] = None,
        return_raw: bool = False,
    ):

        res = await job_routes.get_jobs(
            auth=auth or self.auth,
            application_id=application_id or self.id,
            debug_api=debug_api,
            session=session,
        )
        if debug_api:
            print("Getting Domostats jobs")

        if res.status == 200 and not return_raw:
            self.jobs = [dmdj.DomoJob._from_json(job) for job in res.response]
            return self.jobs

        if res.status == 200 and return_raw:
            return res.response

    async def get_all_schedules(self, auth: dmda.DomoFullAuth = None):
        if not self.jobs and (self.auth or auth):
            await self.get_jobs()

        elif not self.jobs and not (self.auth or auth):
            raise Exception("pass a auth object")

        schedules = pd.DataFrame(
            [
                {
                    "hour": trigger.schedule.hour,
                    "minute": trigger.schedule.minute,
                    "job_id": job.id,
                    "job_name": job.name,
                    "trigger_id": trigger.id,
                }
                for job in self.jobs
                for trigger in job.triggers
            ]
        )

        self.jobs_schedule = schedules.sort_values(
            ["hour", "minute"], ascending=True
        ).reset_index(drop=True)
        return self.jobs_schedule

    async def find_next_job_schedule(
        self, auth: dmda.DomoAuth = None
    ) -> dmdj.DomoTrigger_Schedule:
        if not isinstance(self.jobs_schedule, pd.DataFrame) and (self.auth or auth):
            await self.get_all_schedules()

        elif not isinstance(self.jobs_schedule, pd.DataFrame) and not (
            self.auth or auth
        ):
            raise Exception("pass a auth object")

        df_all_hours = pd.DataFrame(range(0, 23), columns=["hour"])
        df_all_minutes = pd.DataFrame(range(0, 60), columns=["minute"])

        df_all_hours["tmp"] = 1
        df_all_minutes["tmp"] = 1
        df_all = pd.merge(df_all_hours, df_all_minutes, on="tmp").drop(columns=["tmp"])

        # get the number of occurencies of each hour and minutes
        schedules_grouped = (
            self.jobs_schedule.groupby(["hour", "minute"])
            .size()
            .reset_index(name="cnt_schedule")
        )

        # print(schedules_grouped)
        # print(df_all)

        schedules_interpolated = pd.merge(
            df_all, schedules_grouped, how="left", on=["hour", "minute"]
        )

        schedules_interpolated["cnt_schedule"] = schedules_interpolated[
            "cnt_schedule"
        ].fillna(value=0)
        schedules_interpolated.sort_values(
            ["cnt_schedule", "hour", "minute"], ascending=True, inplace=True
        )

        schedules_interpolated.reset_index(drop=True, inplace=True)

        return dmdj.DomoTrigger_Schedule(
            hour=int(schedules_interpolated.loc[0].get("hour")),
            minute=int(schedules_interpolated.loc[0].get("minute")),
        )